# Import

In [1]:
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import os
import copy

from kesacco import clustpipe

# Parameters

### Define the simulation

In [2]:
cpipe = clustpipe.ClusterPipe(silent=False, output_dir='/Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev')

             _  __  ___    __    __     ___   ___   __         
            | |/ / | __| /' _/  /  \   / _/  / _/  /__\        
            |   <  | _|  `._`. | /\ | | \__ | \__ | \/ |       
            |_|\_\ |___| |___/ |_||_|  \__/  \__/  \__/        
Keen Event Simulation and Analysis for CTA Cluster Observations
---------------------------------------------------------------


### Define the cluster object

In [3]:
# Set the cluster basic properties
cpipe.cluster.name     = 'Perseus'
cpipe.cluster.redshift = 0.017284
cpipe.cluster.M500     = 6.2e14*u.solMass # Used for getting R500 and the pressure profile
cpipe.cluster.coord    = SkyCoord("3h19m47.2s +41d30m47s", frame='icrs')

# Truncate the cluster at 1.5 deg to allow small map field of view (for faster code) without loss of flux
cpipe.cluster.theta_truncation = 1.5*u.deg

# The target gas density [Churazov et al. 2003]
cpipe.cluster.density_gas_model = {'name':'doublebeta', 'beta1':1.2, 'r_c1':57*u.kpc, 'n_01':0.046*u.cm**-3,
                                   'beta2':0.71, 'r_c2':278*u.kpc, 'n_02':0.0036*u.cm**-3}

# The thermal profile (assuming Planck 2013 UPP)
cpipe.cluster.set_pressure_gas_gNFW_param('P13UPP')

# CR physics
cpipe.cluster.X_cre1_E = {'X':0.0, 'R_norm':cpipe.cluster.R500}
cpipe.cluster.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.2}
cpipe.cluster.set_density_crp_isobaric_scal_param(scal=1.0)
cpipe.cluster.X_crp_E = {'X':0.2, 'R_norm':cpipe.cluster.R500}

# Sampling
cpipe.cluster.map_reso = 0.01*u.deg      # Ideally should be few times smaller than the PSF
cpipe.cluster.Npt_per_decade_integ = 30

# Get information about the state of the cluster model
cpipe.cluster.print_param()

--- silent
    True
    <class 'bool'>
--- output_dir
    /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev
    <class 'str'>
--- cosmo
    FlatLambdaCDM(name="Planck15", H0=67.7 km / (Mpc s), Om0=0.307, Tcmb0=2.725 K, Neff=3.05, m_nu=[0.   0.   0.06] eV, Ob0=0.0486)
    <class 'astropy.cosmology.core.FlatLambdaCDM'>
--- name
    Perseus
    <class 'str'>
--- coord
    <SkyCoord (ICRS): (ra, dec) in deg
    (49.94666667, 41.51305556)>
    <class 'astropy.coordinates.sky_coordinate.SkyCoord'>
--- redshift
    0.017284
    <class 'float'>
--- D_ang
    74.8907830317466 Mpc
    <class 'astropy.units.quantity.Quantity'>
--- D_lum
    77.50198024167615 Mpc
    <class 'astropy.units.quantity.Quantity'>
--- M500
    620000000000000.0 solMass
    <class 'astropy.units.quantity.Quantity'>
--- R500
    1317.548394363056 kpc
    <class 'astropy.units.quantity.Quantity'>
--- theta500
    1.0080007077672122 deg
    <class 'astropy.units.quantity.Quantity'>
--- R_truncation
    1.9606361149510187 Mpc

### Define the point source object

In [4]:
name='NGC1275'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h19m48.16s +41d30m42s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h19m48.16s +41d30m42s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':2.1e-11/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-3.6, 'free':True},
                   'PivotEnergy':{'value':0.2*u.TeV, 'free':False}}}

cpipe.compact_source.add_source(name, spatial, spectral) # Add the source to the model

In [5]:
name='IC310'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h16m42.98s +41d19m30s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h16m42.98s +41d19m30s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':4.3e-12/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-1.95, 'free':True},
                   'PivotEnergy':{'value':1.0*u.TeV, 'free':False}}}

cpipe.compact_source.add_source(name, spatial, spectral)# Add the source to the model

In [6]:
# Show the status of the compact sources in the sky model
cpipe.compact_source.print_source()

--- NGC1275
    -- Spatial model: PointSource
         RA: {'value': <Longitude 49.95066667 deg>, 'free': False}
         DEC: {'value': <Latitude 41.51166667 deg>, 'free': False}
    -- Spectral model: PowerLaw
         Prefactor: {'value': <Quantity 2.1e-11 1 / (cm2 s TeV)>, 'free': True}
         Index: {'value': -3.6, 'free': True}
         PivotEnergy: {'value': <Quantity 0.2 TeV>, 'free': False}
    -- Temporal model: Constant
         Normalization: {'value': 1.0, 'free': False}
--- IC310
    -- Spatial model: PointSource
         RA: {'value': <Longitude 49.17908333 deg>, 'free': False}
         DEC: {'value': <Latitude 41.325 deg>, 'free': False}
    -- Spectral model: PowerLaw
         Prefactor: {'value': <Quantity 4.3e-12 1 / (cm2 s TeV)>, 'free': True}
         Index: {'value': -1.95, 'free': True}
         PivotEnergy: {'value': <Quantity 1. TeV>, 'free': False}
    -- Temporal model: Constant
         Normalization: {'value': 1.0, 'free': False}


### Define the observations

In [7]:
# One pointing offset +0 +1
cpipe.obs_setup.add_obs(obsid='001', name='Perseus_Ptg1', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+0, cpipe.cluster.coord.dec.value+1, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-01T00:00:00.0', tmax='2020-01-01T01:00:00.0', deadc=0.95)

# One pointing offset +0 -1
cpipe.obs_setup.add_obs(obsid='002', name='Perseus_Ptg2', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+0, cpipe.cluster.coord.dec.value-1, unit='deg'),
                        rad=5*u.deg, 
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-02T00:00:00.0', tmax='2020-01-02T01:00:00.0', deadc=0.95)

# One pointing offset +1 +0
cpipe.obs_setup.add_obs(obsid='003', name='Perseus_Ptg3', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+1, cpipe.cluster.coord.dec.value+0, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-03T00:00:00.0', tmax='2020-01-03T01:00:00.0', deadc=0.95)

# One pointing offset -1 +0
cpipe.obs_setup.add_obs(obsid='004', name='Perseus_Ptg4', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value-1, cpipe.cluster.coord.dec.value+0, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-04T00:00:00.0', tmax='2020-01-04T01:00:00.0', deadc=0.95)

# Print info
cpipe.obs_setup.print_obs()

=== Perseus_Ptg1, ObsID 001
        RA-Dec:    49.94666666666666, 42.51305555555555 deg
        GLON-GLAT: 150.0078784158764, -12.426877669678154 deg
        ROI rad: 5.0 deg
        tmin: 2020-01-01T00:00:00.0
        tmax: 2020-01-01T01:00:00.0
        emin: 0.05 TeV
        emax: 100.0 TeV
        deadc: 0.95
        caldb: prod3b-v2
        irf: North_z20_S_5h
        bkg: name Background, obsid None, spatial type CTAIrfBackground, spectral type PowerLaw
=== Perseus_Ptg2, ObsID 002
        RA-Dec:    49.94666666666666, 40.51305555555555 deg
        GLON-GLAT: 151.14097159102025, -14.095338103823554 deg
        ROI rad: 5.0 deg
        tmin: 2020-01-02T00:00:00.0
        tmax: 2020-01-02T01:00:00.0
        emin: 0.05 TeV
        emax: 100.0 TeV
        deadc: 0.95
        caldb: prod3b-v2
        irf: North_z20_S_5h
        bkg: name Background, obsid None, spatial type CTAIrfBackground, spectral type PowerLaw
=== Perseus_Ptg3, ObsID 003
        RA-Dec:    50.94666666666666, 41.5130

# Run the simulation

In [8]:
cpipe.run_sim_obs()
cpipe.run_sim_quicklook(ShowEvent=True, ShowObsDef=True, ShowSkyModel=True, 
                        bkgsubtract='NONE', smoothing_FWHM=0.1*u.deg)

----- ObsID to be observed: ['001', '002', '003', '004']
------- Simulation log -------
=== GApplication ===
 Name ......................: ctobssim
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Sim_ObsDef.xml
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Sim_Model_Unstack.xml
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outevents .................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Events.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/TmpEvents
 startindex ................: 1
 seed ......................: 857048
 ra ........................: 83.63
 dec .......................: 22.51
 rad .......................: 5.0
 tmin ......................: 2020-01-01T00:00:00
 tmax ......................: 2020-01-01T00:30:00
 mjdref ....................: 51544.5
 emin .............

----- Quicklook 3/4 done
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Events004.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Sim_Skymap004.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 501
 nypix .....................: 501
 binsz .....................: 0.01
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.04
 inradius ..................: 1.00800070776721
 outradius .................: 1.20960084932065
 iterations ................: 3
 thr

# Run the analysis

In [9]:
#----- Analysis parameters
cpipe.method_binned = True    # Do a binned analysis
cpipe.method_stack  = True    # Stack the event from different observations in a single analysis?
cpipe.method_ana    = '3D'    # 3D or ONOFF analysis
cpipe.spec_enumbins = 30
cpipe.spec_emin     = 50*u.GeV
cpipe.spec_emax     = 10*u.TeV
cpipe.spec_edisp    = False

# Force the use of the user defined map grid
cpipe.map_UsePtgRef     = True
# Define the map used for the binned analysis
cpipe.map_reso          = 0.05*u.deg # Can be increaded if the code is too slow
cpipe.map_fov           = 3*u.deg    # Can also be reduced (but should increase bkg-cluster degeneracy)
cpipe.map_coord         = copy.deepcopy(cpipe.cluster.coord)
# Define the map used for the template
cpipe.cluster.map_fov   = 2.1*cpipe.cluster.theta_truncation
cpipe.cluster.map_coord = copy.deepcopy(cpipe.cluster.coord)

In [10]:
cpipe.run_ana_dataprep()


            Starting the data preparation             

----- ObsID to be analysed: ['001', '002', '003', '004']

=== GApplication ===
 Name ......................: ctselect
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Events.xml
 outobs ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_EventsSelected.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Selected
 usepnt ....................: no
 ra ........................: 49.9466666666667
 dec .......................: 41.5152212787165
 rad .......................: 2.33651517298212
 forcesel ..................: yes
 tmin ......................: NONE
 tmax ......................: NONE
 emin ......................: 0.05
 emax ......................: 10
 phase .....................: NONE
 expr ......................: 
 usethres ..................: NONE
 publish ...................: no
 chatter ............

(<gammalib.model.GModels; proxy of <Swig Object of type 'GModels *' at 0x1a21f04db0> >,
 <ctools.tools.ctbin; proxy of <Swig Object of type 'ctbin *' at 0x1a244ad4b0> >,
 <ctools.tools.ctbin; proxy of <Swig Object of type 'ctbin *' at 0x1a21f64d80> >,
 <ctools.tools.ctexpcube; proxy of <Swig Object of type 'ctexpcube *' at 0x1a2a2f5390> >,
 <ctools.tools.ctpsfcube; proxy of <Swig Object of type 'ctpsfcube *' at 0x1092065d0> >,
 <ctools.tools.ctbkgcube; proxy of <Swig Object of type 'ctbkgcube *' at 0x1a21edd390> >)

In [11]:
cpipe.run_ana_likelihood()


              Starting the likelihood fit             

=== GApplication ===
 Name ......................: ctlike
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Model_Input_Stack.xml
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Psfcube.fits
 edispcube .................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Edispcube.fits
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Bkgcube.fits
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outmodel ..................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Model_Output.xml
 outcovmat .................: /Users/adam/Project

(<ctools.tools.ctlike; proxy of <Swig Object of type 'ctlike *' at 0x1a2ab7ccf0> >,
 <ctools.tools.ctmodel; proxy of <Swig Object of type 'ctmodel *' at 0x1a29c676f0> >,
 <ctools.tools.ctmodel; proxy of <Swig Object of type 'ctmodel *' at 0x1a2ab7c7e0> >)

In [12]:
cpipe.run_ana_imaging()


            Starting the imaging analysis             

=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_EventsSelected.xml
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_SkymapTot.fits
 emin ......................: 0.05
 emax ......................: 10
 usepnt ....................: no
 nxpix .....................: 67
 nypix .....................: 67
 binsz .....................: 0.05
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5152212787165
 bkgsubtract ...............: NONE
 roiradius .................: 0.04
 inradius ..................: 1.00800070776721
 outradius .................: 1.20960084932065

In [13]:
cpipe.run_ana_spectral()


            Starting the spectral analysis            

--- Computing spectrum: Perseus
=== GApplication ===
 Name ......................: csspec
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Model_Output.xml
 srcname ...................: Perseus
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Bkgcube.fits
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outfile ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Spectrum_Perseus.fits
 method ....................: SLICE
 ebina

In [14]:
cpipe.run_ana_plot()

----- ObsID to be looked at: ['001', '002', '003', '004']

=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_SelectedEvents001.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Skymap001.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 101
 nypix .....................: 101
 binsz .....................: 0.05
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: NONE
 roiradius .................: 0.04
 inradius ..................: 1.00800070776721
 outradius .................: 1.20960

In [ ]:
cpipe.run_ana_mcmc_spectralimaging(reset_modelgrid=True,
                                   reset_mcmc=True, run_mcmc=True, GaussLike=False,
                                   spatial_range=[0.0,2.0], spatial_npt=3, spectral_range=[2.0,3.0], spectral_npt=3,
                                   bkg_marginalize=False,
                                   bkg_spectral_npt=13, bkg_spectral_range=[-0.05,0.05],
                                   ps_spectral_npt=19, ps_spectral_range=[-1.0,1.0],
                                   rm_tmp=True,
                                   FWHM=0.1*u.deg,
                                   theta=1.0*u.deg,
                                   coord=None,
                                   profile_reso=0.05*u.deg,
                                   includeIC=True)


      Starting the MCMC spectral imaging analysis     
      (including the background in the grid model)

--- Building cluster template 1/9
--- Building cluster template 2/9
--- Building cluster template 3/9
--- Building cluster template 4/9
--- Building cluster template 5/9
--- Building cluster template 6/9
--- Building cluster template 7/9
--- Building cluster template 8/9
--- Building cluster template 9/9
--- Building background template 1/13
--- Building background template 2/13
--- Building background template 3/13
--- Building background template 4/13
--- Building background template 5/13
--- Building background template 6/13
--- Building background template 7/13
--- Building background template 8/13
--- Building background template 9/13
--- Building background template 10/13
--- Building background template 11/13
--- Building background template 12/13
--- Building background template 13/13
------ Point source NGC1275
--- Building point source template 1/13
--- Building point s

/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


nwalkers should be at least twice the number of parameters.
nwalkers --> 18

--- MCMC profile parameters: 
    ndim                = 9
    nwalkers            = 18
    nsteps              = 1000
    burnin              = 100
    conf                = 68.0
    reset_mcmc          = True
    Gaussian likelihood = False
--- No pre-existing sampler, start from scratch
--- Runing 1000 MCMC steps
